In [275]:
import os
import numpy
import pandas
import lightgbm
os.getcwd()

'C:\\mmo\\PycharmProjects\\M5Forecasting\\Notebook'

In [4]:
path = "C:\\mmo\\PycharmProjects\\M5Forecasting\\Data"
files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if '.csv' in file:
            files.append(os.path.join(r, file))

for f in files:
    print(f)

C:\mmo\PycharmProjects\M5Forecasting\Data\calendar.csv
C:\mmo\PycharmProjects\M5Forecasting\Data\sales_train_validation.csv
C:\mmo\PycharmProjects\M5Forecasting\Data\sample_submission.csv
C:\mmo\PycharmProjects\M5Forecasting\Data\sell_prices.csv


In [76]:
sub = pandas.read_csv("C:\mmo\PycharmProjects\M5Forecasting\Data\sample_submission.csv")
sub.head(5)

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### Calendar --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [218]:
caldtype = {"date": "object",
             "wm_yr_wk": 'int16',
             "weekday": "category",
             "wday": "int16",
             "month": "int16", 
             "year": "int16", 
             "d": "object", 
             "event_name_1": "category",
             "event_type_1": "category",  
             "event_name_2": "category", 
             "event_type_2": "category", 
             "snap_CA": "float32", 
             'snap_TX': 'float32', 
             'snap_WI': 'float32' 
            }

In [219]:
cal = pandas.read_csv("C:\mmo\PycharmProjects\M5Forecasting\Data\calendar.csv", dtype = caldtype)
cal["date"] = pandas.to_datetime(cal["date"])

In [220]:
cal.dtypes

date            datetime64[ns]
wm_yr_wk                 int16
weekday               category
wday                     int16
month                    int16
year                     int16
d                       object
event_name_1          category
event_type_1          category
event_name_2          category
event_type_2          category
snap_CA                float32
snap_TX                float32
snap_WI                float32
dtype: object

 #### Price ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [222]:
pricedtype = {"store_id": "category", 
               "item_id": "category", 
               "wm_yr_wk": "int16",
               "sell_price":"float32" 
             }

In [223]:
price = pandas.read_csv("C:\mmo\PycharmProjects\M5Forecasting\Data\sell_prices.csv", dtype = pricedtype)
price.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


 #### Sales ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [224]:
salesnumcols = [f"d_{day}" for day in range(1, 1914)]
salescatcols = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
saledtype = {numcol:"float32" for numcol in salesnumcols} 
saledtype.update({col: "category" for col in salescatcols if col != "id"})

In [225]:
sales = pandas.read_csv("C:\mmo\PycharmProjects\M5Forecasting\Data\sales_train_validation.csv", dtype=saledtype)
sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,1.0,3.0,0.0,1.0,1.0,1.0,3.0,0.0,1.0,1.0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,2.0,1.0,2.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,4.0,1.0,0.0,1.0,3.0,7.0,2.0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,2.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,2.0,4.0


 #### Merge Data --------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [226]:
#merge dfs
df = pandas.melt(sales,
                 id_vars = salescatcols,
                 value_vars = salesnumcols,
                 var_name = "d",
                 value_name = "salesqty")

df = df.merge(cal, on = "d", copy = False)
df = df.merge(price, on = ["store_id", "item_id", "wm_yr_wk"], copy = False)

# category to numeric 
for col in df.select_dtypes("category"):
    df[col] =  df[col].cat.codes.astype("int16")

In [238]:
df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,salesqty,date,wm_yr_wk,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_008_CA_1_validation,7,0,0,0,0,d_1,12.0,2011-01-29,11101,...,1,2011,0,0,0,0,0.0,0.0,0.0,0.46
1,HOBBIES_1_008_CA_1_validation,7,0,0,0,0,d_2,15.0,2011-01-30,11101,...,1,2011,0,0,0,0,0.0,0.0,0.0,0.46
2,HOBBIES_1_008_CA_1_validation,7,0,0,0,0,d_3,0.0,2011-01-31,11101,...,1,2011,0,0,0,0,0.0,0.0,0.0,0.46
3,HOBBIES_1_008_CA_1_validation,7,0,0,0,0,d_4,0.0,2011-02-01,11101,...,2,2011,0,0,0,0,1.0,1.0,0.0,0.46
4,HOBBIES_1_008_CA_1_validation,7,0,0,0,0,d_5,0.0,2011-02-02,11101,...,2,2011,0,0,0,0,1.0,0.0,1.0,0.46


 #### Additional Features 

In [244]:
%%time
# lag and rolling mean features
lags = [7, 28]
lag_cols = [f"lag_{lag}" for lag in lags ]
for lag, lag_col in zip(lags, lag_cols):
    df[lag_col] = df[["id","salesqty"]].groupby("id")["salesqty"].shift(lag)

wins = [7, 28]
for win in wins:
    for lag,lag_col in zip(lags, lag_cols):
        df[f"rmean_{lag}_{win}"] = df[["id", lag_col]].groupby("id")[lag_col].transform(lambda x : x.rolling(win).mean())

#calendar features        
calfeat = {
            "week": "weekofyear",
            "quarter": "quarter",
            "mday": "day",
          }
for col, func in calfeat.items():
    df[col] = getattr(df["date"].dt, func).astype("int16")

Wall time: 2min 14s


In [258]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46027957 entries, 0 to 46027956
Data columns (total 31 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            object        
 1   item_id       int16         
 2   dept_id       int16         
 3   cat_id        int16         
 4   store_id      int16         
 5   state_id      int16         
 6   d             object        
 7   salesqty      float32       
 8   date          datetime64[ns]
 9   wm_yr_wk      int16         
 10  weekday       int16         
 11  wday          int16         
 12  month         int16         
 13  year          int16         
 14  event_name_1  int16         
 15  event_type_1  int16         
 16  event_name_2  int16         
 17  event_type_2  int16         
 18  snap_CA       float32       
 19  snap_TX       float32       
 20  snap_WI       float32       
 21  sell_price    float32       
 22  lag_7         float32       
 23  lag_28        float32       
 

In [256]:
df.shape

(46027957, 31)

**lag_7**: sales shifted 7 steps downwards for each group. The example above focuses on one group only as an example. That is why the first value appears on the 7th index.  
**lag_28**: sales shifted 28 steps downwads. That is why the first value appears on the 28th index.  
**rmean_7_7**: rolling mean sales of a window size of 7 over column lag_7. First value (0.2857) appears on the 13th index because means including nan are nan.  
**rmean_7_28**: rolling mean sales of a window size of 7 over column lag_28. First value (0.357) appears on the 34th index because that is the first time the mean formula gets all 7 non-nan values.  
**rmean_28_7**: rolling mean sales of a window size of 28 over column lag_7. First value (0.2857) appears on the 3th index because it is the first time the mean formula gets 28 non-nan values.  
**rmean_28_28**: rolling mean sales of a window size of 28 over column lag_28. First value appears on 55th index because that is the first time the formula here all non-nan values. 


The intuition as far as I can understand is the following:


1. Captures the week-on-week similarity and that too of just the past week. In other words, people are likely to shop this monday similar to the last monday (except it is some special occassion).
2. Captures the weekly similarity from a month-to-month perspective. Example: people in the 1st weekend of a month shop more so that weekend looks more similar to first weeks of other months than the previous weekend. (Though 28 is arguable here. A month is generally 30. Interesting would be a variable window depending on when the comparative week starts. Dealing with edge cases like week divided into 2 months will be tricky).
3. Captures the information regarding the sales of the whole previous week ending 7 days in the past i.e. if we are at day 14, then the average is of sales from days 1-7 NOT days 7-14. This provides the information about the whole week and not just a single day sale comparison like lag_7 to bring the lag_7 value into "better weekly context".
4. Captures the information regarding the sales of the entire previous 4 weeks ending 7 days in the past i.e. if we are at day 35, then the average is sales from days 1-28.
5. Captures the information regarding the sales of the whole week ending 4 weeks ago i.e. if we are on day 35, then the average is of sales from day 1-7. (Assuming for simplicity the month is 28 days), this provides the information of not just a month-to-month comparison of the same day (day 7 of month one vs day 7 of month two), but the entire week leading up to day 7. Again the idea I believe is to capture the whole week and not just a single day sale comparison like lag_28 to bring the lag_28 value into "better weekly context".
6. Captures the information regarding the sales of the entire previous 4 weeks ending 4 weeks in the past i.e. if we are at day 56, then the average is of days 1-28. (Assuming for simplicity the month is 28 days), the idea again is to bring the point value of lag_28 into a better context (i.e. of day 28 when being compared to day 56) into a "better monthly context".

How would you "talk" about these features?

Hey let's see how the sales were last friday compared to this friday?
Hey let's see how the sales were first weekend of the last month compared to first weekend of this month?
May be comparing last saturday to this saturday is too specific. Week-on-week same day trends are more likely to be similar if the prior week went similar too. It would make sense to not just have the last saturday but also the mean of the whole week leading upto that day to give the model the "hint" how normal the whole week was.

In [259]:
df.dropna(inplace = True)
df.shape

(44351007, 31)

In [260]:
df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,salesqty,date,wm_yr_wk,...,sell_price,lag_7,lag_28,rmean_7_7,rmean_28_7,rmean_7_28,rmean_28_28,week,quarter,mday
617364,HOBBIES_1_008_CA_1_validation,7,0,0,0,0,d_56,0.0,2011-03-25,11108,...,0.42,0.0,0.0,2.000000,4.000000,1.642857,4.535714,12,1,25
617371,HOBBIES_1_009_CA_1_validation,8,0,0,0,0,d_56,0.0,2011-03-25,11108,...,1.77,0.0,0.0,2.000000,2.000000,2.142857,2.178571,12,1,25
617378,HOBBIES_1_010_CA_1_validation,9,0,0,0,0,d_56,0.0,2011-03-25,11108,...,3.17,0.0,0.0,0.000000,0.142857,0.035714,0.214286,12,1,25
617385,HOBBIES_1_012_CA_1_validation,11,0,0,0,0,d_56,1.0,2011-03-25,11108,...,6.27,1.0,1.0,0.428571,0.857143,0.535714,0.678571,12,1,25
617392,HOBBIES_1_015_CA_1_validation,14,0,0,0,0,d_56,1.0,2011-03-25,11108,...,0.72,2.0,1.0,5.428571,8.000000,6.142857,4.321429,12,1,25


In [281]:
cat_feats = ["item_id", "dept_id", "cat_id", "store_id", "state_id"] + \
            ["event_name_1", "event_name_2", "event_type_1", "event_type_2"]
useless_cols = ["id", "date", "salesqty","d", "wm_yr_wk", "weekday"]
train_cols = df.columns[~df.columns.isin(useless_cols)]

X_train = df[train_cols]
y_train = df["salesqty"]

In [282]:
train_data = lightgbm.Dataset(X_train, label = y_train, categorical_feature=cat_feats, free_raw_data=False)

In [283]:
fake_valid_inds = numpy.random.choice(len(X_train), 1000000)
fake_valid_data = lightgbm.Dataset(X_train.iloc[fake_valid_inds], label = y_train.iloc[fake_valid_inds],categorical_feature=cat_feats,
                             free_raw_data=False)

In [284]:
params = {
            "objective" : "poisson",
            "metric" :"rmse",
            "force_row_wise" : True,
            "learning_rate" : 0.075,
           #"sub_feature" : 0.8,
            "sub_row" : 0.75,
            "bagging_freq" : 1,
            "lambda_l2" : 0.1,
           #"nthread" : 4
            "metric": ["rmse"],
            "verbosity": 1,
           #"num_iterations" : 200,
            "num_iterations" : 2500,
}

In [285]:
%%time
model_lgb = lightgbm.train(params, train_data, valid_sets = [fake_valid_data], verbose_eval=100)

MemoryError: Unable to allocate 1.65 GiB for an array with shape (10, 44351007) and data type float32